In [17]:
import pandas as pd
import numpy as np
df3 = pd.read_csv('agencycmoprices.csv')

#get rid of stars
df3.replace('*', '', inplace=True)
df3.replace('0', '', inplace=True)
df3['Date'] = pd.to_datetime(df3['Date'])
vintages = ['Pre-2009', '2009-2013', '2014-2016', 'Post-2016']
df3[vintages] = df3[vintages].apply(pd.to_numeric)
df3 = df3.replace(np.NaN, '')
df3[df3['Measure'] == 'AVERAGE PRICE']['Measure2'].unique()
df3[df3['Measure'] == "VOLUME OF TRADES (000'S)"]['Measure2'].unique()

temp = df3[df3['Measure'] == 'AVERAGE PRICE']
temp = temp[temp['Measure2'] == 'TOTAL']
temp = temp[temp['MortgageType'] == 'AGENCY CMO P&I']
temp = temp.set_index(['Date', 'Agency']).unstack(level=1)['Pre-2009']
temp

Agency,FHLMC,FNMA,GNMA
Date,,,
2019-01-02,101,101.2,103.7
2019-01-03,104,101.5,105
2019-01-04,101.5,101,103
2019-01-07,102.6,102.5,105.3
2019-01-08,101.5,99.5,103.4
2019-01-09,101.4,103.5,104.2
2019-01-10,103.4,104.2,103.1
2019-01-11,100.4,102.8,101.1
2019-01-14,101.7,101.2,103.2
